# Feature Selection and LASSO Interpretation

In this notebook, we will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though we can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

# Fire up Graphlab Create

In [94]:
import graphlab
import numpy as np

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [95]:
sales = graphlab.SFrame('kc_house_data.gl/')

# Create new features

In [96]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [97]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [7]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 0.893847     | 6869667.895833     | 385998.458623 |

| 6         | 7        | 0.000002  | 0.958652     | 6847177.773672     | 380824.455891 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

## Finding what features has non-zero weight.

In [9]:
model_all.get('coefficients').print_rows(num_rows = 20)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None |  274873.05595 |  None  |
|     bedrooms     |  None |      0.0      |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 8468.53108691 |  None  |
|   sqft_living    |  None | 24.4207209824 |  None  |
| sqft_living_sqrt |  None | 350.060553386 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 842.068034898 |  None  |
|    sqft_above    |  None | 20.0247224171 |  None  |
|  sqft_basement   |  None |

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a **Subset Selection**.

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

In [10]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When we call `linear_regression.create()` then we have to make sure you set `validation_set = None`.

We can also turn off the print out of `linear_regression.create()` with `verbose = False`

# Function to compute RSS of all the models using different L1_penalty

In [46]:
def input_penalty_compute_RSS(training, validation, l1_penalty, all_features):
    RSS = {}
    for penalty in l1_penalty:
        model = graphlab.linear_regression.create(training, features = all_features, target = 'price',  validation_set=None,
                                                  verbose = False, l1_penalty = penalty, l2_penalty = 0.0)
        predictions = model.predict(validation)
        rss = sum((predictions - validation['price'])**2)
        RSS[penalty] = rss
    return RSS

# Creating different L1_penalty and then calculating RSS on them

In [98]:
l1_penalty = np.logspace(1, 7, num = 13)
RSS = input_penalty_compute_RSS(training, validation, l1_penalty, all_features)

In [48]:
RSS

{10.0: 625766285142461.2,
 31.622776601683793: 625766285362395.2,
 100.0: 625766286057886.9,
 316.2277660168379: 625766288257224.9,
 1000.0: 625766295212185.9,
 3162.2776601683795: 625766317206077.8,
 10000.0: 625766386760661.6,
 31622.776601683792: 625766606749281.4,
 100000.0: 625767302791633.2,
 316227.7660168379: 625769507643884.9,
 1000000.0: 625776517727025.6,
 3162277.660168379: 625799062845466.9,
 10000000.0: 625883719085424.5}

# Computing min RSS of the model evaluated on different L1_penalty

In [44]:
penalty_min_rss = min(RSS.items(), key = lambda x : x[1])

In [45]:
penalty_min_rss

(10.0, 625766285142461.2)

# Checking sparsity of weights on above calculated penalty having min RSS

In [23]:
model_penalty_10 = graphlab.linear_regression.create(training, features = all_features, target = 'price', validation_set=None,
                                              verbose = False, l1_penalty = 10.0, l2_penalty = 0.0)
model_penalty_10.get('coefficients').print_rows(num_rows = 20)

+------------------+-------+------------------+--------+
|       name       | index |      value       | stderr |
+------------------+-------+------------------+--------+
|   (intercept)    |  None |  18993.4272128   |  None  |
|     bedrooms     |  None |  7936.96767903   |  None  |
| bedrooms_square  |  None |  936.993368193   |  None  |
|    bathrooms     |  None |  25409.5889341   |  None  |
|   sqft_living    |  None |  39.1151363797   |  None  |
| sqft_living_sqrt |  None |  1124.65021281   |  None  |
|     sqft_lot     |  None | 0.00348361822299 |  None  |
|  sqft_lot_sqrt   |  None |  148.258391011   |  None  |
|      floors      |  None |   21204.335467   |  None  |
|  floors_square   |  None |  12915.5243361   |  None  |
|    waterfront    |  None |  601905.594545   |  None  |
|       view       |  None |  93312.8573119   |  None  |
|    condition     |  None |  6609.03571245   |  None  |
|      grade       |  None |  6206.93999188   |  None  |
|    sqft_above    |  None |  4

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, we are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region we found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

# Setting max number of features we want to having in our model

In [24]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [25]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, we are going to implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * We are going to fit a regression model with a given `l1_penalty` on TRAIN data. We are going to Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When we call `linear_regression.create()` we are going to make sure to set `validation_set = None`
    * We are going to extract the weights of the model and count the number of nonzeros. Then, we will save the number of nonzeros to a list. Using `model['coefficients']['value']` which gives us an SArray with the parameters we learned. If we call the method `.nnz()` on it, we will find the number of non-zero parameters!* 

# Calulating the models and their count of non-zero weights

In [34]:
nnz_weights = {}
for penalty in l1_penalty_values:
    model = graphlab.linear_regression.create(training, features = all_features, target = 'price',  validation_set=None,
                                              verbose = False, l1_penalty = penalty, l2_penalty = 0.0)
    
    nnz_weights[penalty] = model['coefficients']['value'].nnz()

In [35]:
nnz_weights

{100000000.0: 18,
 127427498.57031322: 18,
 162377673.91887242: 18,
 206913808.111479: 18,
 263665089.87303555: 17,
 335981828.6283788: 17,
 428133239.8719396: 17,
 545559478.1168514: 17,
 695192796.1775591: 17,
 885866790.4100832: 16,
 1128837891.6846883: 15,
 1438449888.2876658: 15,
 1832980710.8324375: 13,
 2335721469.0901213: 12,
 2976351441.631313: 10,
 3792690190.7322536: 6,
 4832930238.571753: 5,
 6158482110.6602545: 3,
 7847599703.514623: 1,
 10000000000.0: 1}

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)

There are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [38]:
l1_penalty_min = 2976351441.631313
l1_penalty_max = 3792690190.7322536

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [39]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * We will fit a regression model with a given `l1_penalty` on TRAIN data. Then, specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When we call `linear_regression.create()` we have to make sure to set `validation_set = None`
    * We will measure the RSS of the learned model on the VALIDATION set
 
The we will find the model that has lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

# Function to return RSS of all models & all models too

In [59]:
def return_rss_model(training, validation, l1_penalty, all_features):
    RSS, model_collection = {}, {}
    for penalty in l1_penalty:
        model = graphlab.linear_regression.create(training, features = all_features, target = 'price',  validation_set=None,
                                                  verbose = False, l1_penalty = penalty, l2_penalty = 0.0)
        predictions = model.predict(validation)
        rss = sum((predictions - validation['price'])**2)
        RSS[penalty] = rss
        model_collection[penalty] = model
    return RSS, model_collection

# Computing RSS and collection of all models

In [60]:
RSS_range, model_collection = return_rss_model(training, validation, l1_penalty_values, all_features)

# Finding model will sparsity = max_nonzeros and minimum RSS

In [90]:
def compute_rss_and_compare_sparsity(RSS_range, model_collection, max_nonzeros):
    sorted_rss = dict(sorted(RSS_range.items(), key = lambda x : x[1]))
    for penalty, rss in sorted_rss.items():
        if model_collection[penalty]['coefficients']['value'].nnz() == max_nonzeros:
            return penalty, model_collection[penalty]

In [91]:
l1_penalty_having_max_nonzeros, sparse_model = compute_rss_and_compare_sparsity(RSS_range, model_collection, max_nonzeros)

In [92]:
l1_penalty_having_max_nonzeros, RSS_range[l1_penalty_having_max_nonzeros]

(3424044711.4391184, 1044917964832727.1)

# Chekcing the weight of above selected model having minimum RSS and sparsity = max_nonzeros

In [99]:
sparse_model.get('coefficients').print_rows(num_rows = 20)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 220905.678163 |  None  |
|     bedrooms     |  None |  750.87574334 |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 15999.1708621 |  None  |
|   sqft_living    |  None | 32.5179051834 |  None  |
| sqft_living_sqrt |  None | 696.113506892 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 2939.65709214 |  None  |
|    sqft_above    |  None | 30.1732109884 |  None  |
|  sqft_basement   |  None |

# Good Luck!!!